In [ ]:
library(data.table)
library(mlr3misc)
library(lme4)
library(MuMIn)
library(lmerTest)
library(dplyr)
library(stringr)

In [ ]:
latex_table_glmm = function(model) {
  coefs = summary(model)$coefficients
  coefs_df = as.data.frame(coefs)
  
  coefs_df$Predictor = rownames(coefs_df)
  
  format_p = function(p) {
    if (p < 0.001) {
      return("$<0.001$")
    } else {
      return(sprintf("$%.3f$", p))
    }
  }
  
  latex_rows = coefs_df %>%
    mutate(
      Estimate = sprintf("%.6f", Estimate),
      `Std. Error` = sprintf("%.6f", `Std. Error`),
      `z value` = sprintf("%.3f", `z value`),
      `Pr(>|z|)` = sapply(`Pr(>|z|)`, format_p)
    ) %>%
    transmute(
      line = sprintf("    %s & %s & %s & %s & %s \\\\", 
                     Predictor, Estimate, `Std. Error`, `z value`, `Pr(>|z|)`))
    
  cat("\\begin{tabular}{lrrrr}\n")
  cat("    \\toprule\n")
  cat("    Predictor & Estimate & Std. Error & $z$ value & $p$-value \\\\\n")
  cat("    \\midrule\n")
  cat(paste(latex_rows$line, collapse = "\n"), "\n")
  cat("    \\bottomrule\n")
  cat("\\end{tabular}\n")
}

latex_table_lmm = function(model) {
  coefs = summary(model)$coefficients
  coefs_df = as.data.frame(coefs)
  
  coefs_df$Predictor = rownames(coefs_df)
  
  format_p = function(p) {
    if (p < 0.001) {
      return("$<0.001$")
    } else {
      return(sprintf("$%.3f$", p))
    }
  }
  
  latex_rows = coefs_df %>%
    mutate(
      Estimate = sprintf("%.3e", Estimate),
      `Std. Error` = sprintf("%.3e", `Std. Error`),
      df = sprintf("%.3e", df),
      `t value` = sprintf("%.3f", `t value`),
      `Pr(>|t|)` = sapply(`Pr(>|t|)`, format_p)
    ) %>%
    transmute(
      line = sprintf("    %s & %s & %s & %s & %s & %s \\\\",
                     Predictor, Estimate, `Std. Error`, df, `t value`, `Pr(>|t|)`))
  
  cat("\\begin{tabular}{lrrrrr}\n")
  cat("    \\toprule\n")
  cat("    Predictor & Estimate & Std. Error & df & $t$ value & $p$-value \\\\\n")
  cat("    \\midrule\n")
  cat(paste(latex_rows$line, collapse = "\n"), "\n")
  cat("    \\bottomrule\n")
  cat("\\end{tabular}\n")
}

In [ ]:
files = dir("csvs/")
files = files[sapply(files, grepl, pattern = "reshuffling")]
data = map_dtr(paste0("csvs/", files), function(x) {
  tmp = fread(x)
  tmp[, c("seed", "data_id", "repeat", "metric", "resampling", "reshuffled", "retrained", "classifier", "optimizer", "train_valid_size",
          "overtuning", "overtuning_relative", "iteration", "final_iteration", "no_progress", "meta_overfitting", "test_regret")]
}, .fill = TRUE)
data = data[metric != "balanced_accuracy"]
data[no_progress == TRUE, overtuning_relative := 0]
factors = c("seed", "data_id", "repeat", "metric", "resampling", "reshuffled", "retrained", "classifier", "optimizer", "train_valid_size")
data[, (factors) := lapply(.SD, as.factor), .SDcols = factors]
data[, metric := relevel(metric, ref = "accuracy")]
data[, resampling := relevel(resampling, ref = "holdout")]
data[, reshuffled := relevel(reshuffled, ref = "FALSE")]
data[, retrained := relevel(retrained, ref = "TRUE")]
data[, classifier := relevel(classifier, ref = "logreg")]
data[, optimizer := relevel(optimizer, ref = "random")]
data[, overtuning_nonzero := overtuning > 1e-3]
data[, log_overtuning := log(overtuning)]
data[, log_overtuning_relative := log(overtuning_relative)]

In [ ]:
# random search retrained not reshuffled
data1 = data[iteration %in% seq(0, 500, by = 10)[-1] & retrained == TRUE & reshuffled == FALSE & optimizer == "random", ]
data1[, budget := iteration / 500]

# predict probability of non-zero overtuning
glmm1_0 = glmer(overtuning_nonzero ~ (1 | data_id) + (1 | seed) + budget + metric + classifier + resampling + train_valid_size, data = data1, family = binomial)
glmm1_1 = glmer(overtuning_nonzero ~ (1 | data_id) + (1 | seed) + budget + I(budget^2) + metric + classifier + resampling + train_valid_size, data = data1, family = binomial)
anova(glmm1_0, glmm1_1)

# for non-zero overtuning model log_overtuning_relative
lmm1_0 = lmer(log_overtuning_relative ~ (1 | data_id) + (1 | seed) + budget + metric + classifier + resampling + train_valid_size, data = data1[overtuning_nonzero == TRUE & no_progress == FALSE, ])
lmm1_1 = lmer(log_overtuning_relative ~ (1 | data_id) + (1 | seed) + budget + I(budget^2) + metric + classifier + resampling + train_valid_size, data = data1[overtuning_nonzero == TRUE & no_progress == FALSE, ])
anova(lmm1_0, lmm1_1)

# meta overfitting and test regret
of_lmm1 = lmer(meta_overfitting ~ (1 | data_id) + (1 | seed) + metric + classifier + resampling + train_valid_size, data = data1[iteration == 500, ])
tr_lmm1 = lmer(test_regret ~ (1 | data_id) + (1 | seed) + metric + classifier + resampling + train_valid_size, data = data1[iteration == 500, ])
rm(data1)

In [ ]:
latex_table_glmm(glmm1_1)
latex_table_lmm(lmm1_1)
latex_table_lmm(of_lmm1)
latex_table_lmm(tr_lmm1)

In [ ]:
# bo retrained not reshuffled
data2 = data[iteration %in% seq(0, 250, by = 10)[-1] & metric == "auc" & retrained == TRUE & reshuffled == FALSE & optimizer != "hebo_makarova"]
data2[, budget := iteration / 250]

# predict probability of non-zero overtuning
glmm2_0 = glmer(overtuning_nonzero ~ (1 | data_id) + (1 | seed) + budget + I(budget^2) + classifier + resampling + train_valid_size, data = data2, family = binomial)
glmm2_1 = glmer(overtuning_nonzero ~ (1 | data_id) + (1 | seed) + budget + I(budget^2) + classifier + resampling + train_valid_size + optimizer, data = data2, family = binomial)
anova(glmm2_0, glmm2_1)

# for non-zero overtuning model log_overtuning_relative
lmm2_0 = lmer(log_overtuning_relative ~ (1 | data_id) + (1 | seed) + budget + I(budget^2) + classifier + resampling + train_valid_size, data = data2[overtuning_nonzero == TRUE & no_progress == FALSE, ])
lmm2_1 = lmer(log_overtuning_relative ~ (1 | data_id) + (1 | seed) + budget + I(budget^2) + classifier + resampling + train_valid_size + optimizer, data = data2[overtuning_nonzero == TRUE & no_progress == FALSE, ])
anova(lmm2_0, lmm2_1)

# meta overfitting and test regret
of_lmm2 = lmer(meta_overfitting ~ (1 | data_id) + (1 | seed) + classifier + resampling + train_valid_size + optimizer, data = data2[iteration == 250, ])
tr_lmm2 = lmer(test_regret ~ (1 | data_id) + (1 | seed) + classifier + resampling + train_valid_size + optimizer, data = data2[iteration == 250, ])
rm(data2)

In [ ]:
latex_table_glmm(glmm2_1)
latex_table_lmm(lmm2_1)
latex_table_lmm(of_lmm2)
latex_table_lmm(tr_lmm2)

In [ ]:
# hebo vs. hebo hebo makarova early stopping cv retrained not reshuffled
data3 = data[final_iteration == TRUE & metric == "auc" & retrained == TRUE & reshuffled == FALSE & optimizer %in% c("hebo", "hebo_makarova") & resampling == "cv"]

# predict probability of non-zero overtuning
glmm3_0 = glmer(overtuning_nonzero ~ (1 | data_id) + (1 | seed) + classifier + train_valid_size, data = data3, family = binomial)
glmm3_1 = glmer(overtuning_nonzero ~ (1 | data_id) + (1 | seed) + classifier + train_valid_size + optimizer, data = data3, family = binomial)
anova(glmm3_0, glmm3_1)

# for non-zero overtuning model log_overtuning_relative
lmm3_0 = lmer(log_overtuning_relative ~ (1 | data_id) + (1 | seed) + classifier + train_valid_size, data = data3[overtuning_nonzero == TRUE & no_progress == FALSE, ])
lmm3_1 = lmer(log_overtuning_relative ~ (1 | data_id) + (1 | seed) + + classifier + train_valid_size + optimizer, data = data3[overtuning_nonzero == TRUE & no_progress == FALSE, ])
anova(lmm3_0, lmm3_1)

# meta overfitting and test regret
of_lmm3 = lmer(meta_overfitting ~ (1 | data_id) + (1 | seed) + classifier + train_valid_size + optimizer, data = data3[iteration == 250, ])
tr_lmm3 = lmer(test_regret ~ (1 | data_id) + (1 | seed) + classifier + train_valid_size + optimizer, data = data3[iteration == 250, ])
rm(data3)

In [ ]:
latex_table_glmm(glmm3_1)
latex_table_lmm(lmm3_1)
latex_table_lmm(of_lmm3)
latex_table_lmm(tr_lmm3)

In [ ]:
# random search retrained, reshuffled vs. not
data4 = data[iteration %in% seq(0, 500, by = 10)[-1] & retrained == TRUE & optimizer == "random", ]
data4[, budget := iteration / 500]

# predict probability of non-zero overtuning
glmm4_0 = glmer(overtuning_nonzero ~ (1 | data_id) + (1 | seed) + budget + I(budget^2) + metric + classifier + resampling + train_valid_size, data = data4, family = binomial)
glmm4_1 = glmer(overtuning_nonzero ~ (1 | data_id) + (1 | seed) + budget + I(budget^2) + metric + classifier + resampling + train_valid_size + reshuffled, data = data4, family = binomial)
anova(glmm4_0, glmm4_1)

# for non-zero overtuning model log_overtuning_relative
lmm4_0 = lmer(log_overtuning_relative ~ (1 | data_id) + (1 | seed) + budget + I(budget^2) + metric + classifier + resampling + train_valid_size, data = data4[overtuning_nonzero == TRUE & no_progress == FALSE, ])
lmm4_1 = lmer(log_overtuning_relative ~ (1 | data_id) + (1 | seed) + budget + I(budget^2) + metric + classifier + resampling + train_valid_size + reshuffled, data = data4[overtuning_nonzero == TRUE & no_progress == FALSE, ])
anova(lmm4_0, lmm4_1)

# meta overfitting and test regret
of_lmm4 = lmer(meta_overfitting ~ (1 | data_id) + (1 | seed) + metric + classifier + resampling + train_valid_size + reshuffled, data = data4[iteration == 500, ])
tr_lmm4 = lmer(test_regret ~ (1 | data_id) + (1 | seed) + metric + classifier + resampling + train_valid_size + reshuffled, data = data4[iteration == 500, ])
rm(data4)

In [ ]:
latex_table_glmm(glmm4_1)
latex_table_lmm(lmm4_1)
latex_table_lmm(of_lmm4)
latex_table_lmm(tr_lmm4)

In [ ]:
# random search retrained, reshuffled vs. not, holdout auc
data5 = data[iteration %in% seq(0, 500, by = 10)[-1] & retrained == TRUE & optimizer == "random" & resampling == "holdout" & metric == "auc", ]
data5[, budget := iteration / 500]

# predict probability of non-zero overtuning
glmm5_0 = glmer(overtuning_nonzero ~ (1 | data_id) + (1 | seed) + budget + I(budget^2) + classifier + train_valid_size, data = data5, family = binomial)
glmm5_1 = glmer(overtuning_nonzero ~ (1 | data_id) + (1 | seed) + budget + I(budget^2) + classifier + train_valid_size + reshuffled, data = data5, family = binomial)
anova(glmm5_0, glmm5_1)

# for non-zero overtuning model log_overtuning_relative
lmm5_0 = lmer(log_overtuning_relative ~ (1 | data_id) + (1 | seed) + budget + I(budget^2) + classifier + train_valid_size, data = data5[overtuning_nonzero == TRUE & no_progress == FALSE, ])
lmm5_1 = lmer(log_overtuning_relative ~ (1 | data_id) + (1 | seed) + budget + I(budget^2) + classifier + train_valid_size + reshuffled, data = data5[overtuning_nonzero == TRUE & no_progress == FALSE, ])
anova(lmm5_0, lmm5_1)

# meta overfitting and test regret
of_lmm5 = lmer(meta_overfitting ~ (1 | data_id) + (1 | seed) + classifier + train_valid_size + reshuffled, data = data5[iteration == 500, ])
tr_lmm5 = lmer(test_regret ~ (1 | data_id) + (1 | seed) + classifier + train_valid_size + reshuffled, data = data5[iteration == 500, ])
rm(data5)

In [ ]:
latex_table_glmm(glmm5_1)
latex_table_lmm(lmm5_1)
latex_table_lmm(of_lmm5)
latex_table_lmm(tr_lmm5)

In [ ]:
# random search not reshufled, retrained vs. not retrained
data6 = data[iteration %in% seq(0, 500, by = 10)[-1] & reshuffled == FALSE & optimizer == "random", ]
data6[, budget := iteration / 500]

# predict probability of non-zero overtuning
glmm6_0 = glmer(overtuning_nonzero ~ (1 | data_id) + (1 | seed) + budget + I(budget^2) + metric + classifier + resampling + train_valid_size, data = data6, family = binomial)
glmm6_1 = glmer(overtuning_nonzero ~ (1 | data_id) + (1 | seed) + budget + I(budget^2) + metric + classifier + resampling + train_valid_size + retrained, data = data6, family = binomial)
anova(glmm6_0, glmm6_1)

# for non-zero overtuning model log_overtuning_relative
lmm6_0 = lmer(log_overtuning_relative ~ (1 | data_id) + (1 | seed) + budget + I(budget^2) + metric + classifier + resampling + train_valid_size, data = data6[overtuning_nonzero == TRUE & no_progress == FALSE, ])
lmm6_1 = lmer(log_overtuning_relative ~ (1 | data_id) + (1 | seed) + budget + I(budget^2) + metric + classifier + resampling + train_valid_size + retrained, data = data6[overtuning_nonzero == TRUE & no_progress == FALSE, ])
anova(lmm6_0, lmm6_1)

# meta overfitting and test regret
of_lmm6 = lmer(meta_overfitting ~ (1 | data_id) + (1 | seed) + metric + classifier + resampling + train_valid_size + retrained, data = data6[iteration == 500, ])
tr_lmm6 = lmer(test_regret ~ (1 | data_id) + (1 | seed) + metric + classifier + resampling + train_valid_size + retrained, data = data6[iteration == 500, ])
rm(data6)

In [ ]:
latex_table_glmm(glmm6_1)
latex_table_lmm(lmm6_1)
latex_table_lmm(of_lmm6)
latex_table_lmm(tr_lmm6)